In [1]:
# 对偏移数据进行加权平均

In [2]:
import numpy as np 
import cv2 as cv
import queue, threading, time 
from configparser import ConfigParser
from numpy.core.numeric import zeros_like
import matplotlib.pyplot as plt
import colorsys
import math
import random
from vidgear.gears import VideoGear
from vidgear.gears import WriteGear
import datetime
import os
import csv

from mpl_toolkits.mplot3d import Axes3D

#hsv transform to rgb format
def hsv2rgb(h,s,v):
    return tuple(round(i * 255) for i in colorsys.hsv_to_rgb(h,s,v))

In [3]:
#control parameters
is_save_stream = False
is_anti_shake = False

# default parameters  
# calculation parameters
winsize = 64
mag_ceiling = 0.8
mag_floor = 0 # mag_ceiling/5

# camer & image parameters
alpha = 0.8 
hue = 90                     

# # 方案一
# options = {"CAP_PROP_FRAME_WIDTH":2592, "CAP_PROP_FRAME_HEIGHT":1944, "CAP_PROP_FPS":30}
# RES=(2592, 1944)                 # camera resolution
# roi_rect = [200,RES[0]-200,50,RES[1]-600]  # region of interesting

# 方案二
roi_rect = [20,780,20,580] # 转置前
RES=(800, 600)
options = {"CAP_PROP_FRAME_WIDTH":800, "CAP_PROP_FRAME_HEIGHT":600, "CAP_PROP_FPS":30}

result_RES = (640, 480)         # flow image resolution
# stable calculation parameters
pyr_scale = 0.5
levels = 2
iterations = 1
poly_n = 5
poly_sigma = 1.1
flags = cv.OPTFLOW_FARNEBACK_GAUSSIAN
# Noffset = 30 # 消除光流算法偏差的位移量
Noffset = 0
# drawing buffer
ax = []
ay1 = []
ay2 = []


In [4]:
# 水平噪声、数值噪声
stream = VideoGear(source=0, stabilize=is_anti_shake, **options).start() # To open any valid video stream(for e.g device at 0 index)

# pre-read frame
frame_pre = stream.read()
print("video resolution is (height, width, channel) : ",frame_pre.shape)
bgr_pre = cv.cvtColor(frame_pre, cv.COLOR_BGR2GRAY)

# initial hsv
hsv = np.zeros_like(frame_pre[roi_rect[2]+Noffset:roi_rect[3]-Noffset,roi_rect[0]+Noffset:roi_rect[1]-Noffset])  
# hsv = np.zeros_like(frame_pre[roi_rect[2]:roi_rect[3],roi_rect[0]:roi_rect[1]])  
hsv[...,1] = 255 #saturation is full
blank = np.zeros_like(frame_pre)

# video loop
count = 0
print("begin stream!")
print("video resolution is (height, width, channel) : ",frame_pre.shape)
# 程序执行区域
timeNow = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
os.makedirs("tVariance_" + timeNow)
with open("tVariance_" + timeNow + '/Variance.csv','w',newline='')as csv_file:
    while(True):
    # read current frame from cap
        frame_cur = stream.read()
        bgr_cur = cv.cvtColor(frame_cur, cv.COLOR_BGR2GRAY) # change in to gray
        
        is_ref_refresh = False
        # 测量两幅同样图像
        if is_ref_refresh:
            bgr_pre = bgr_cur
        # 图像裁切
        roi_bgr_pre = bgr_pre[roi_rect[2]:roi_rect[3],roi_rect[0]:roi_rect[1]]
        roi_bgr_cur = bgr_cur[roi_rect[2]:roi_rect[3],roi_rect[0]:roi_rect[1]]

        # calculate flow
        
        flow_frame = cv.calcOpticalFlowFarneback(roi_bgr_pre,roi_bgr_cur, None,
                                        pyr_scale,
                                        levels,
                                        winsize,
                                        iterations,
                                        poly_n,
                                        poly_sigma,
                                        flags)
        if count == 0:
            flow = np.zeros_like(flow_frame)
        # mag_frame 和 ang_frame 都是每幅图像的矢量，加到
        # flow = (flow + flow_frame)/2

        flow = (flow*count + flow_frame)/(count+1)

        # flow = flow_frame
        
        mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])  # orginal flow
        mag = mag[Noffset:mag.shape[0]-Noffset, Noffset:(mag.shape[1]-Noffset)]
        ang = ang[Noffset:ang.shape[0]-Noffset, Noffset:(ang.shape[1]-Noffset)]



        mag_mean = cv.mean(mag)[0]
        mag_sft = abs(mag - mag_mean)  # shifted magnitude to elimiate noise
        hsv[...,0] = (ang + hue)*180/np.pi/2 # color space related to angle 
        hsv[...,2] = cv.normalize(mag_sft,None,0,255,cv.NORM_MINMAX) 
        

        #enhanced flow
        bgr_flow = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
        mag_enhanced = zeros_like(mag_sft)
        cv.min(mag_sft,mag_ceiling,mag_enhanced)  # enhance flow, ceiling and flooring
        cv.max(mag_enhanced,mag_floor,mag_enhanced)
        hsv[...,0] = (ang + hue)*180/np.pi/2 # color space related to angle 
        hsv[...,2] = cv.normalize(mag_enhanced,None,0,255,cv.NORM_MINMAX)
        bgr_flow_enhanced = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
        
        # image emerge with enhanced flow
        flow_blend_enhance = cv.addWeighted(frame_cur[roi_rect[2]+Noffset:roi_rect[3]-Noffset,roi_rect[0]+Noffset:roi_rect[1]-Noffset], 1-alpha ,bgr_flow_enhanced,  alpha, 0)
        # bgr_blend = cv.addWeighted(frame_cur[roi_rect[2]+Noffset:roi_rect[3]-Noffset,roi_rect[0]+Noffset:roi_rect[1]-Noffset], 1-alpha ,bgr_flow,  alpha, 0)
        # bgr_blend = bgr_blend[0:bgr_blend.shape[0], Noffset:bgr_blend.shape[0]-Noffset]
        frame_blend = cv.addWeighted(frame_cur, 1-alpha ,blank,  alpha, 0)#frame_cur.copy()
        frame_blend[roi_rect[2]+Noffset:roi_rect[3]-Noffset,roi_rect[0]+Noffset:roi_rect[1]-Noffset] = flow_blend_enhance

        #  # image emerge with enhanced flow
        # flow_blend_enhance = cv.addWeighted(frame_cur[roi_rect[2]:roi_rect[3],roi_rect[0]-Noffset:roi_rect[1]], 1-alpha ,bgr_flow_enhanced,  alpha, 0)
        # bgr_blend = cv.addWeighted(frame_cur[roi_rect[2]:roi_rect[3],roi_rect[0]:roi_rect[1]], 1-alpha ,bgr_flow,  alpha, 0)
        # bgr_blend = bgr_blend[0:bgr_blend.shape[0], Noffset:bgr_blend.shape[0]-Noffset]
        # frame_blend = cv.addWeighted(frame_cur, 1-alpha ,blank,  alpha, 0)#frame_cur.copy()
        # frame_blend[roi_rect[2]:roi_rect[3],roi_rect[0]:roi_rect[1]] = flow_blend_enhance

        # if ture, compare image is  successive; if false, compare image is not change
        if is_ref_refresh:
            bgr_pre = bgr_cur
        
        #palette
        palette=np.zeros((512,512,3),np.uint8)
        mag_norm = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
        mag_min,mag_max,min_indx,max_indx=cv.minMaxLoc(mag)
        ang_mean = cv.mean(ang*mag/mag_mean)[0]

        # 程序结果的问题出在这几行
        # 对结果矩阵偏移的修正
        horMat = np.multiply(mag, np.cos(ang))
        verMat = np.multiply(mag, np.sin(ang))
        
        # horMat = horMat - 0.12*np.ones(horMat.shape)
        # 计算水平噪声最大值和竖直噪声最大值
        _,hor_MaxNoise, _, _ = cv.minMaxLoc(np.multiply(mag, np.cos(ang))-0*np.ones(horMat.shape))
        _,ver_MaxNoise, _, _ = cv.minMaxLoc(np.multiply(mag, np.sin(ang)))

        # 计算水平噪声和竖直噪声
        hor_Noise = np.average(np.multiply(mag, np.cos(ang))-0*np.ones(horMat.shape))
        ver_Noise = np.average(np.multiply(mag, np.sin(ang)))
        # # 绝对值计算水平噪声和竖直噪声
        # hor_AbsNoise = np.average(np.abs(horMat))
        # ver_AbsNoise = np.average(np.abs(verMat))


        cv.putText(palette, "max="+str(mag.max()), (0, 30), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
        cv.putText(palette, "mag_mean="+str("%.7f"%mag_mean), (0, 60), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
        cv.putText(palette, "mag_max="+str(mag_max), (0, 90), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
        cv.putText(palette, "mag_min="+str(mag_min), (0, 120), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
        cv.putText(palette, "hor_Noise="+str(hor_Noise), (0, 150), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
        cv.putText(palette, "ver_Noise="+str(ver_Noise), (0, 180), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
        cv.putText(palette, "hor_MaxNoise="+str(hor_MaxNoise), (0, 210), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
        cv.putText(palette, "ver_MaxNoise="+str(ver_MaxNoise), (0, 240), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)

        timeNow1 = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        # max min min variance 
        # 获取一个csv对象进行内容写入
        writer=csv.writer(csv_file)
        # writerow 写入一行数据
        writeDatum = [timeNow1, mag_mean, mag_max, mag_min, hor_Noise, ver_Noise, hor_MaxNoise, ver_MaxNoise]
        writer.writerow(writeDatum)

        count = count + 1
    
        # result flow image
        cv.imshow('Orignal',cv.resize(frame_cur, (result_RES[0], result_RES[1])))
        cv.imshow("Palette", palette)
        # cv.imshow('Blender', cv.resize(bgr_blend , (result_RES[0], result_RES[1])))
        cv.imshow('Area of Intrest Blended', cv.resize(frame_blend  , (result_RES[0], result_RES[1])))
        
        key = cv.waitKey(10)
        if key==114: # if input key 'r', refresh compare image
            #打印时间戳保存
            # timeNow1 = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            # os.makedirs("Refresh_" + timeNow1)
            bgr_pre = bgr_cur
            # cv.imwrite(timeNow +'/Orignal.jpg',frame_cur)
            # cv.imwrite(timeNow +'/Area_of_Intrest_Blended.jpg', frame_blend)
            # cv.imwrite(timeNow1 +"/Palette.jpg", palette)
            # cv.imwrite(timeNow1 +'/Blender.jpg', bgr_blend)
            # print("the image is refreshed")

video resolution is (height, width, channel) :  (600, 800, 3)
begin stream!
video resolution is (height, width, channel) :  (600, 800, 3)


In [23]:
import numpy as np
import cv2
hsv = np.zeros([600, 540, 3]).astype(np.uint8)

# for x in range(600):
#     for y in range(180):
#         hsv[x,y*3,0]=y
#         hsv[x,y*3+1,0]=y
#         hsv[x,y*3+2,0]=y

hsv[...,1] = 255 #saturation is full
hsv[...,2] = 255
bgr_flow_enhanced = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
cv2.imshow("1", bgr_flow_enhanced)
cv2.waitKey()

-1

In [1]:
import numpy as np
import cv2
hsv = np.zeros([600, 540, 3]).astype(np.uint8)
# 生成色相矩阵
m1 = np.arange(0,180,1)
m1 = m1.reshape(-1, 1)
m2 = np.multiply(m1, np.array([1,1,1]))
m3 = m2.flatten().reshape(1, -1)
m4 = np.ones([600, 1])
m5 = np.multiply(m4, m3).astype(np.uint8)
# 生成hsv矩阵
hsv[...,0] = m5
hsv[...,1] = 255 #saturation is full
hsv[...,2] = 255
pic = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)



cv2.imshow("1", pic)
cv2.waitKey()

-1

In [2]:
import numpy as np
import cv2
L = 600
halfL = int(L/2)
x = range(-halfL ,halfL)
y = range(-halfL ,halfL)
X, Y = np.meshgrid(x, y)
mag, ang = cv2.cartToPolar(X.astype(np.float), Y.astype(np.float))
# 生成mask掩膜,利用色值来进行掩膜操作
mask = np.zeros_like(mag).astype(np.uint8)
mask[np.where(mag<halfL-100)] = 255

# 生成ang色环
hsv = np.zeros([L, L, 3])
hsv[...,0] = (ang)*180/2/np.pi
hsv[...,1] = 255 #saturation is full
hsv[...,2] = mask
# 掩膜生成
pic = cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)

cv2.imshow("1", pic)
cv2.waitKey()

d:\Users\lenovo\Anaconda3\envs\school\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


-1